In [1]:
import pickle 
import nltk
import pertubations_helper as ph

with open('ne_collection.obj', 'rb') as f:
    ne_collection = pickle.load(f)

with open('pos_collection.obj', 'rb') as f:
    pos_collection = pickle.load(f)

In [2]:
from datasets import load_dataset

squad = load_dataset('squad_v2')

/Users/v/miniconda/envs/mthesis/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset squad_v2 (/Users/v/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 194.31it/s]


In [3]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/bert-small-finetuned-squadv2",
    tokenizer="mrm8488/bert-small-finetuned-squadv2"
)

qa_pipeline({
    'context': "Manuel Romero has been working hardly in the repository hugginface/transformers lately",
    'question': "Who has been working hard for hugginface/transformers lately?"

})

{'score': 0.9939450621604919, 'start': 0, 'end': 13, 'answer': 'Manuel Romero'}

In [4]:
import spacy
from spacy import displacy

ner = spacy.load('en_core_web_sm')

In [5]:
# import numpy as np

# n = 32

# beam_width = 2

# for i, q_ctx in enumerate(squad['train']):
#     question = q_ctx['question']
#     context = q_ctx['context']
    
#     score1 = qa_pipeline({'context': context, 'question': question})

#     question_chunks = question.split()

#     while True:
#         idxs = [ random.randint(0, len(question_chunks) - 1) for i in range(beam_width) ]
#         tokens = [ nltk.word_tokenize( question_chunks[idx] ) for idx in idxs ]    
#         tags = [ nltk.pos_tag(token)[0] for token in tokens ]

#         real_tags = [ tag[1] for tag in tags ]

#         if all( np.asarray(real_tags) == real_tags[0] ):
#             continue
        
#         if all([ tag[1] not in ph.excluded_POS_tags for tag in tags ]):
#             break

#     subs = {}
#     for tag in tags:
#         substitution = random.choices(pos_collection[tag[1]], k=n)
#         subs[tag[1]] = substitution

#     k1, k2 = subs.keys()
#     v1, v2 = subs.values()

#     best_score = score1
#     best_question = ''
#     for i, v1_ in enumerate(v1):
#         new_question_chunks = question_chunks.copy()
#         new_question_chunks[idxs[0]] = v1[i]
#         for j, v2_ in enumerate(v2):
#             new2_question_chunks = new_question_chunks.copy()
#             new2_question_chunks[idxs[1]] = v2[j]
#             new_question = ' '.join(new2_question_chunks)
#             score2 = qa_pipeline({'context': context, 'question': new_question})
            
#             if score2['score'] > best_score['score'] and \
#                ph.is_undersensitivity_attack( score1, score2 ):

#                 best_score = score2
#                 best_question = new_question
#                 print( f'{question}({round(score1["score"],2)}) vs {new_question}({round(score2["score"],2)}) => {score1["answer"]}')

#     print( f'Best score: {best_score["score"]}, {best_question=}' )

#     break
        

In [6]:
# import numpy as np

# n = 32

# beam_width = 3

# for i, q_ctx in enumerate(squad['train']):
#     question = q_ctx['question']
#     context = q_ctx['context']
    
#     score1 = qa_pipeline({'context': context, 'question': question})

#     question_chunks = question.split()

#     while True:
#         idxs = [ random.randint(0, len(question_chunks) - 1) for i in range(beam_width) ]
#         tokens = [ nltk.word_tokenize( question_chunks[idx] ) for idx in idxs ]    
#         tags = [ nltk.pos_tag(token)[0] for token in tokens ]

#         real_tags = [ tag[1] for tag in tags ]

#         if all( np.asarray(real_tags) == real_tags[0] ):
#             continue
        
#         if all([ tag[1] not in ph.excluded_POS_tags for tag in tags ]):
#             break

#     subs = []
#     for tag in tags:
#         substitution = random.choices(pos_collection[tag[1]], k=n)
#         subs.append( substitution )

#     subs = np.asarray(subs)

#     indexes = [0] * beam_width

#     best_score = score1.copy()
#     best_score['score'] = 0.0

#     best_question = ''

#     for i in range(n**beam_width):
#         new_question_chunks = question_chunks.copy()
#         for j, idx in enumerate(indexes):
#             new_question_chunks[idxs[j]] = subs[j][idx]
        
#         new_question = ' '.join(new_question_chunks)
#         score2 = qa_pipeline({'context': context, 'question': new_question})

#         if score2['score'] > best_score['score'] and \
#               ph.is_undersensitivity_attack( score1, score2 ):
#             best_score = score2
#             best_question = new_question
#             print( f'{question}({round(score1["score"],2)}) vs {new_question}({round(score2["score"],2)}) => {score1["answer"]}')

#         for j in reversed(range(beam_width)):
#             indexes[j] += 1
#             if indexes[j] == n:
#                 indexes[j] = 0
#             else:
#                 break

#         if all(np.asarray(indexes) == 0):
#             print(indexes)
#             break
        
#     break
        

In [7]:
import statistics

print( statistics.mean( [len(ne_collection[k]) for k in ne_collection]) )
print( statistics.mean( [len(pos_collection[k]) for k in pos_collection]) )

5094.055555555556
5091.84


In [13]:
import numpy as np

n = 32

beam_size = 1

for i, q_ctx in enumerate(squad['train']):
    question = q_ctx['question']
    context = q_ctx['context']
    
    score1 = qa_pipeline({'context': context, 'question': question})

    question_chunks = question.split()

    def is_attack_question( new_question ):
        score2 = qa_pipeline({'context': context, 'question': new_question})

        if ph.is_undersensitivity_attack( score1, score2 ):
            print( f'{question}({round(score1["score"],2)}) vs {new_question}({round(score2["score"],2)}) => {score1["answer"]}')
            return True
        
        return False

    new_question = ph.pos_pertubation( question, pos_collection, is_attack_question, beam_size=beam_size, n=n)
    
    break
        

When did Beyonce start becoming popular?(0.38) vs When did Beyonce start ordaining popular?(0.45) => late 1990s


In [16]:
import numpy as np

n = 32

beam_size = 2

new_questions = []
for i, q_ctx in enumerate(squad['train']):
    question = q_ctx['question']
    context = q_ctx['context']
    
    score1 = qa_pipeline({'context': context, 'question': question})

    question_chunks = question.split()

    def is_attack_question( new_question ):
        score2 = qa_pipeline({'context': context, 'question': new_question})

        if ph.is_undersensitivity_attack( score1, score2 ):
            print( f'{question}({round(score1["score"],2)}) vs {new_question}({round(score2["score"],2)}) => {score1["answer"]}')
            return True
        
        return False

    while True:
        new_question = ph.ne_pertubation( question, ne_collection, ner, is_attack_question, beam_size=beam_size, n=n)
        if new_question != '':
            break
        
    new_questions.append(new_question)
    if len(new_questions) == 5:
        break

When did Beyonce start becoming popular?(0.38) vs When did Charterhouse start becoming popular?(0.46) => late 1990s
When did Beyonce leave Destiny's Child and become a solo singer?(0.84) vs When did Paul leave Spears and become a solo singer?(0.87) => 2003
In what city and state did Beyonce  grow up? (0.67) vs In what city and state did Evander Childs  grow up? (0.67) => Houston, Texas
In which decade did Beyonce become famous?(0.79) vs In the next fiscal year did Mandisa become famous?(0.84) => 1990s
